# Let's Go!

## To do:
- Data Understanding
- K-Means
- Classificatore

## Done:
- Droppare ciò che non è Currency == USD 
- Convertire in float: Rating, Rating Count, Price
- Convertire in bool: Editors Choice, In App Purchases, Ad Supported 
- Installs: Togliere i + alla fine e convertire in float 
- Minimum Android: Prendere i primi tre valori (e.g.: 4.2), convertire in float 
- Content Rating: Mettere in scala 0-4, convertire in int
- Size: Togliere M, k, eventualmente G. Tutto deve essere in scala MB. Convertire in float
- Rimuovere nan
- FEATURE: Fixare data
- FEATURE: stdev dalla media per categoria


## Next:
- Sperimentazioni di clustering / classificazioni
- Applicazioni esoteriche


In [1]:
from math import ceil

import numpy as np
import matplotlib.pyplot as plt

from pyspark import SparkContext

from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import col
from pyspark.sql.functions import udf
from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import unix_timestamp
from pyspark.sql import functions as F

from pyspark.sql.types import FloatType
from pyspark.sql.types import BooleanType
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DateType

from datetime import datetime


sc = SparkContext(appName="MY-APP-NAME", master="local[*]")
sc.setLogLevel("ERROR")

In [2]:
# import pandas as pd

# df = pd.read_csv("Google-Playstore.csv")

# df


In [3]:
# df.isna().sum()

In [4]:
#df.Size.unique()

In [5]:
def sizeConverter(value):
    if value is not None:
        if value[len(value)-1:] == 'M':
            value = value.replace('M', '')
            return float(value)
        elif value[len(value)-1:] == 'G':
            value = value.replace('G', '')
            return float(value) * 1024
        elif value[len(value)-1:] == 'k':
            value = value.replace('k', '')
            return float(value) / 1024
    
def timeConverter(value):
    if value is not None:
        if "@" in str(value):
            return None
        else:
            d = datetime.strptime(value, '%b %d, %Y')
            return(d.strftime('%Y-%m-%d'))
        
def getYear(value):
    d = datetime.strptime(value, '%Y-%m-%d')
    return int(d.strftime('%Y'))

def getQuarter(value):
    d = datetime.strptime(value, '%Y-%m-%d')
    return ceil(int(d.strftime('%m'))/3)

def calculate_stdev(group_feat, feat, dataframe):
    avg_rating_feat = dataframe.groupBy(group_feat).agg(F.mean(feat).alias("temp"))
    data_avg = dataframe.join(avg_rating_feat, [group_feat], 'outer')
    dataframe = data_avg.withColumn(str(feat) + "_stdev_from_"+ str(group_feat), (col(feat) - col("temp")))
    dataframe = dataframe.drop("temp")
    return dataframe


## Data Preparation

In [6]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local") \
    .appName("AppName") \
    .getOrCreate()

data = spark.read.option("header",True).option('inferSchema', True).csv("Google-Playstore.csv")



In [7]:
#for col in data.columns:
 #   newcol = '_'.join(col.split(' '))
  #  data = data.withColumnRenamed(col, newcol)

In [8]:
data.printSchema()

root
 |-- App Name: string (nullable = true)
 |-- App Id: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Rating Count: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Minimum Installs: string (nullable = true)
 |-- Maximum Installs: string (nullable = true)
 |-- Free: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Currency: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Minimum Android: string (nullable = true)
 |-- Developer Id: string (nullable = true)
 |-- Developer Website: string (nullable = true)
 |-- Developer Email: string (nullable = true)
 |-- Released: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Privacy Policy: string (nullable = true)
 |-- Ad Supported: string (nullable = true)
 |-- In App Purchases: string (nullable = true)
 |-- Editors Choice: string (nullable = true)
 |-- Scra

In [9]:
data.show()

+----------------------------------+--------------------+-----------------+------+------------+--------+----------------+----------------+----+-----+--------+----+---------------+--------------------+--------------------+--------------------+------------+------------+--------------+--------------------+------------+----------------+--------------+-------------------+
|                          App Name|              App Id|         Category|Rating|Rating Count|Installs|Minimum Installs|Maximum Installs|Free|Price|Currency|Size|Minimum Android|        Developer Id|   Developer Website|     Developer Email|    Released|Last Updated|Content Rating|      Privacy Policy|Ad Supported|In App Purchases|Editors Choice|       Scraped Time|
+----------------------------------+--------------------+-----------------+------+------------+--------+----------------+----------------+----+-----+--------+----+---------------+--------------------+--------------------+--------------------+------------+-----

In [10]:

# Currency
data = data.filter(data.Currency == 'USD')

# Missing values mascherati
data = data.withColumn("Content Rating", regexp_replace(col("Content Rating"), "Unrated", "-1"))
data = data.withColumn("Minimum Android", regexp_replace(col("Minimum Android"), "Varies with device", "0.0"))
data = data.fillna("0.0", subset="Minimum Android")

#data = data.dropna(subset=['Content Rating', 'Minimum Android'])

# Prepariamo Installs e Content Rating alla trasformazione in numeriche
data = data.withColumn("Installs", regexp_replace(col("Installs"), "\+", ""))
data = data.withColumn("Installs", regexp_replace(col("Installs"), ",", ""))
data = data.withColumn("Rating", regexp_replace(col("Rating"), ",", ""))
data = data.withColumn("Size", regexp_replace(col("Size"), ",", "."))

#data = data.withColumn("Content Rating", regexp_replace(col("Content Rating"), "-1", "-1"))
data = data.withColumn("Content Rating", regexp_replace(col("Content Rating"), "Everyone", "0"))
data = data.withColumn("Content Rating", regexp_replace(col("Content Rating"), "Everyone 10+", "1"))
data = data.withColumn("Content Rating", regexp_replace(col("Content Rating"), "Teen", "2"))
data = data.withColumn("Content Rating", regexp_replace(col("Content Rating"), "Mature 17+", "3"))
data = data.withColumn("Content Rating", regexp_replace(col("Content Rating"), "Adults only 18+", "4"))

# Convertiamo in float...
data = data.withColumn("Price", data["Price"].cast(FloatType()).alias("Price"))
data = data.withColumn("Rating", data["Rating"].cast(FloatType()).alias("Rating"))
data = data.withColumn("Rating Count", data["Rating Count"].cast(FloatType()).alias("Rating Count"))
data = data.withColumn("Installs", data["Installs"].cast(FloatType()).alias("Installs"))
data = data.withColumn("Minimum Android", data["Minimum Android"][0:3].cast(FloatType()).alias("Minimum Android"))
data = data.withColumn("Content Rating", data["Content Rating"].cast(IntegerType()).alias("Content Rating"))

#... e in boolean
data = data.withColumn("Editors Choice", data["Editors Choice"].cast(BooleanType()).alias("Editors Choice"))
data = data.withColumn("In App Purchases", data["In App Purchases"].cast(BooleanType()).alias("In App Purchases"))
data = data.withColumn("Ad Supported", data["Ad Supported"].cast(BooleanType()).alias("Ad Supported"))

# Applichiamo sizeConverter e timeConverter
data = data.withColumn("Size", udf(sizeConverter)("Size"))
data = data.withColumn("Released", udf(timeConverter)("Released"))
data = data.withColumn("Last Updated", udf(timeConverter)("Last Updated"))

# Drop di colonne inutili
data = data.drop('Free', 'Privacy Policy', 'Minimum Installs', 'Maximum Installs', 'Scraped Time', 'Currency', 
          'Developer Email', 'Developer Website')

#Drop di missing values
data = data.dropna(how='any')

#Creazione di feature relative all'anno
data = data.withColumn("Year of Release", udf(getYear)("Released"))
data = data.withColumn("Quarter of Release", udf(getQuarter)("Released"))
data = data.withColumn("Year of Last Update", udf(getYear)("Last Updated"))
data = data.drop('Released', 'Last Updated')

#rinominiamo due feat per la nostra sanità mentale
data = data.withColumnRenamed('Content Rating', 'Age')
data = data.withColumnRenamed('Rating Count', 'nReviews')

#nuova feat
data = data.withColumn("Reviews_Installs_Ratio", (col("nReviews") / col("Installs")))
data = data.withColumn("Rating_Reviews_Ratio", (col("Rating") / col("nReviews")))
data = data.na.drop(subset='Reviews_Installs_Ratio')

#feature statistiche
data = calculate_stdev("Category", "Rating", data)
data = calculate_stdev("Category", "nReviews", data)
data = calculate_stdev("Category", "Installs", data)
data = calculate_stdev("Category", "Price", data)
data = calculate_stdev("Category", "Size", data)

data.show()

+-------------+--------------------+--------------------+------+--------+--------+-----+----+---------------+--------------------+---+------------+----------------+--------------+---------------+------------------+-------------------+----------------------+--------------------+--------------------------+----------------------------+----------------------------+-------------------------+------------------------+
|     Category|            App Name|              App Id|Rating|nReviews|Installs|Price|Size|Minimum Android|        Developer Id|Age|Ad Supported|In App Purchases|Editors Choice|Year of Release|Quarter of Release|Year of Last Update|Reviews_Installs_Ratio|Rating_Reviews_Ratio|Rating_stdev_from_Category|nReviews_stdev_from_Category|Installs_stdev_from_Category|Price_stdev_from_Category|Size_stdev_from_Category|
+-------------+--------------------+--------------------+------+--------+--------+-----+----+---------------+--------------------+---+------------+----------------+------

In [11]:
data.count()

2089852

In [12]:
block

NameError: name 'block' is not defined

In [13]:
data.repartition(1).write.json("cleaned_final_v3.json")